In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 67.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.9 MB/s eta 0:00:00


In [3]:
!pip install torchreid

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torchreid: filename=torchreid-0.2.5-py3-none-any.whl size=144346 sha256=fbee21eff4c28818cea2b8c65a6561c8876b90bcf19dff5cf24a968c8fd0234a
  Stored in directory: /root/.cache/pip/wheels/bb/2d/36/816a48465cefd3e58be0317648a4c52ce39ae817f935212099
Successfully built torchreid


In [4]:
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.7 MB/s eta 0:00:00


In [5]:
import json
from google.colab import files
from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas()
import pandas as pd
import os
import tokenizers
import string
import transformers
import torch.nn as nn
from torch.nn import functional as F
import re
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AutoModelForSequenceClassification
from transformers import AutoConfig, AutoModel
from sklearn.model_selection import train_test_split
from torch.utils.data import Subset
import numpy as np
import torch
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader
from transformers import AdamW, get_linear_schedule_with_warmup
from torch.optim import lr_scheduler
from sklearn import model_selection
from sklearn import metrics
from torchreid import utils
from tqdm.autonotebook import tqdm
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping

<ipython-input-5-63b52b00f0ca>:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm_notebook().pandas()


0it [00:00, ?it/s]

/usr/local/lib/python3.10/dist-packages/torchreid/reid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


In [6]:
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.3 MB/s eta 0:00:00


In [7]:
import torch
import nltk
import numpy as np
import evaluate

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [9]:
import random

SEED = 42

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

# Fine-tuning GPT-2

In [38]:
def generate_random_number():
    length = random.randint(1, 254)
    if length == 0:
        return 0
    else:
        return random.randint(10**(length-1), 10**length-1)

In [35]:
generate_random_number()

13

In [37]:
class NumDataset:
    def __init__(self, length, tokenizer):
        self.length = length
        self.tokenizer = tokenizer
        # self.tokenizer.pad_token = tokenizer.eos_token
    
    def __len__(self):
        return self.length

    def __getitem__(self, item):
        x = generate_random_number()
        y = generate_random_number()
        z = x + y

        inputs = f"<startoftext>{x}+{y}={z}<endoftext>"
        input_ids = self.tokenizer(inputs, truncation=True, padding = "max_length", max_length=512)
        target_ids = self.tokenizer(str(z), truncation=True, padding = "max_length", max_length=512)

        return torch.tensor(input_ids['input_ids']), torch.tensor(input_ids['attention_mask']), str(z)

        # return {
        #     "input_ids": input_ids['input_ids'],
        #     "attention_mask": input_ids['attention_mask'], #+ [0]*(1024 - len(input_ids)),
        #     "labels": str(z),
        # }


In [ ]:
a = NumDataset(1, tokenizer=tokenizer)

In [ ]:
a.__getitem__(0)

In [9]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments

In [ ]:
def load_sentiment_dataset(tokenizer):
  train_dataset = NumDataset(10000, tokenizer=tokenizer)
  return train_dataset

In [31]:
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [33]:
model.num_parameters()

124439808

In [ ]:
model_name = 'gpt2'

torch.manual_seed(42)

tokenizer = GPT2Tokenizer.from_pretrained(model_name, bos_token = '<startoftext>',
                                          eos_token='<endoftext>', pad_token = '<pad>')

model = GPT2LMHeadModel.from_pretrained(model_name).to(device)
model.resize_token_embeddings(len(tokenizer))

train_dataset = load_sentiment_dataset(tokenizer)
eval_dataset = load_sentiment_dataset(tokenizer)

training_args = TrainingArguments(
          output_dir='results',
          num_train_epochs = 2,
          load_best_model_at_end=True,
          save_strategy='epoch',
          evaluation_strategy = 'epoch',
          per_device_train_batch_size = 2,
          per_device_eval_batch_size=2,
          warmup_steps=100,
          weight_decay=0.01,
          logging_dir='logs'
      )

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset = eval_dataset,
        data_collator=lambda data: {"input_ids": torch.stack([f[0] for f in data]),
                                    "attention_mask": torch.stack([f[1] for f in data]),
                                    "labels": torch.stack([f[0] for f in data])}
                  )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.575900,1.596573
2,1.605100,1.589399


TrainOutput(global_step=10000, training_loss=1.675508526611328, metrics={'train_runtime': 4268.3087, 'train_samples_per_second': 4.686, 'train_steps_per_second': 2.343, 'total_flos': 5225840640000000.0, 'train_loss': 1.675508526611328, 'epoch': 2.0})

In [ ]:
model.eval()

In [ ]:
# x = generate_random_number()
# y = generate_random_number()
# z = x + y

x = 2
y = 7

promt = f"<startoftext>{x}+{y}="
generated = tokenizer(f'{promt}', return_tensors='pt').input_ids.cuda()
same_outputs = model.generate(generated, do_sample=False, top_p=1, num_return_sequences=0)
predicted_text = tokenizer.decode(same_outputs[0], skip_special_tokens=True)

print(predicted_text)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


2+7=7


# MWP_BERT

In [10]:
from transformers import BertModel, BertTokenizer, AutoModelForMaskedLM, AutoTokenizer

In [10]:
#hfl/chinese-bert-wwm-ext
#invokerliang/MWP-BERT-en

In [ ]:
model = AutoModelForMaskedLM.from_pretrained("hfl/chinese-bert-wwm-ext")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("hfl/chinese-bert-wwm-ext")

In [23]:
model.num_parameters()

102290312

In [24]:
text = '1+2=3 2+3=[MASK]'

In [25]:
import torch

inputs = tokenizer(text, return_tensors="pt")
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> 1+2=3 2+3=4'
'>>> 1+2=3 2+3=2'
'>>> 1+2=3 2+3=3'
'>>> 1+2=3 2+3=5'
'>>> 1+2=3 2+3=1'


In [22]:
def generate_random_number():
    length = random.randint(1, 10)
    if length == 0:
        return 0
    else:
        return random.randint(10**(length-1), 10**length-1)

In [17]:
class NumDataset:
    def __init__(self, length, tokenizer):
        self.length = length
        self.tokenizer = tokenizer
    
    def __len__(self):
        return self.length

    def __getitem__(self, item):
        x = generate_random_number()
        y = generate_random_number()
        z = x + y

        inputs = f"[BOS]{x}+{y}={z}[EOS]"

        input_ids = self.tokenizer(inputs, truncation=True, padding = "max_length", max_length=512)
        return input_ids
        # target_ids = self.tokenizer(str(z), truncation=True, padding = "max_length", max_length=512)

        # return torch.tensor(input_ids['input_ids']), torch.tensor(input_ids['attention_mask']), str(z)

        # return {
        #     "input_ids": input_ids['input_ids'],
        #     "attention_mask": input_ids['attention_mask'], #+ [0]*(1024 - len(input_ids)),
        #     "labels": str(z),
        # }


In [23]:
def load_sentiment_dataset(tokenizer, task = 'train'):
  train_dataset = NumDataset(10000, tokenizer=tokenizer) if task == 'train' else NumDataset(10, tokenizer=tokenizer)
  return train_dataset
  

In [24]:
model_name = 'hfl/chinese-bert-wwm-ext'

torch.manual_seed(42)

tokenizer = AutoTokenizer.from_pretrained(model_name, bos_token = '[BOS]',
                                          eos_token='[EOS]', pad_token = '[PAD]')

model = AutoModelForMaskedLM.from_pretrained(model_name).to(device)
model.resize_token_embeddings(len(tokenizer))

train_dataset = load_sentiment_dataset(tokenizer)
eval_dataset = load_sentiment_dataset(tokenizer, 'eval')

training_args = TrainingArguments(
          output_dir='results',
          num_train_epochs = 2,
          load_best_model_at_end=True,
          save_strategy='epoch',
          evaluation_strategy = 'epoch',
          per_device_train_batch_size = 2,
          per_device_eval_batch_size=2,
          warmup_steps=100,
          weight_decay=0.01,
          logging_dir='logs',
          logging_steps=10
      )

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset = eval_dataset,
        data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=True, mlm_probability=0.15)
        # data_collator=lambda data: {"input_ids": torch.stack([f[0] for f in data]),
        #                             "attention_mask": torch.stack([f[1] for f in data]),
        #                             "labels": torch.stack([f[0] for f in data])}
                  )

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of the model checkpoint at hfl/chinese-bert-wwm-ext were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,3.342900,2.354423
2,3.133700,3.940022


TrainOutput(global_step=10000, training_loss=3.061288352680206, metrics={'train_runtime': 2418.5329, 'train_samples_per_second': 8.269, 'train_steps_per_second': 4.135, 'total_flos': 5263519211520000.0, 'train_loss': 3.061288352680206, 'epoch': 2.0})

In [ ]:
model.eval()

In [44]:
text = '2+1=[MASK]'

In [ ]:
cnt=0

for i in range(100):
  x = generate_random_number()
  y = generate_random_number()
  z = x + y
  text = f'{x}+{y}=[MASK]'
  inputs = tokenizer(text, return_tensors="pt").to(device)
  token_logits = model(**inputs).logits
  # Find the location of [MASK] and extract its logits
  mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
  mask_token_logits = token_logits[0, mask_token_index, :]
  # Pick the [MASK] candidates with the highest logits
  top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()
  for token in top_5_tokens:
    if tokenizer.decode([token]) == z:
      cnt+=1

print(cnt/100)

In [47]:
import torch

inputs = tokenizer(text, return_tensors="pt").to(device)
token_logits = model(**inputs).logits
# Find the location of [MASK] and extract its logits
mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
mask_token_logits = token_logits[0, mask_token_index, :]
# Pick the [MASK] candidates with the highest logits
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> 726+352664=352'
'>>> 726+352664=2'
'>>> 726+352664=54'
'>>> 726+352664=79'
'>>> 726+352664=4'


# flan-t5-xl

In [ ]:
input_ids = tokenizer.encode("Текст для генерации.", return_tensors='pt')
output = model.generate(input_ids=input_ids, max_length=50, do_sample=False)
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)


In [ ]:
!pip install sentencepiece

In [ ]:
!pip install accelerate

In [11]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [88]:
# tokenizer_t5 = AutoTokenizer.from_pretrained("google/flan-t5-xl")
# model_t5 = T5ForConditionalGeneration.from_pretrained("google/flan-t5-xl", device_map="auto")

input_text = "$18+3="
input_ids = tokenizer_t5(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model_t5.generate(input_ids) #, max_length=50, do_sample=False)
print(tokenizer_t5.decode(outputs[0])[6:-4] == '6')
print(tokenizer_t5.decode(outputs[0])[6:-4])

False
8


In [26]:
model_t5.num_parameters()

2849757184

In [100]:
true = []
ans = []
cnt=0

for i in tqdm(range(1000)):
  x = generate_random_number()
  y = generate_random_number()
  z = x + y
  text = f'{x}+{y}='
  input_ids = tokenizer_t5(input_text, return_tensors="pt").input_ids.to("cuda")

  outputs = model_t5.generate(input_ids)
  try:
      ans.append(int(re.findall(r'\d+', tokenizer_t5.decode(outputs[0])[6:-4])[0]))
      true.append(z)
  except:
      pass


  # print(text, tokenizer_t5.decode(outputs[0])[6:-4] == f'{z}')
  # print(tokenizer_t5.decode(outputs[0])[6:-4])

  if tokenizer_t5.decode(outputs[0])[6:-4] == f'{z}':
    cnt+=1

  0%|          | 0/1000 [00:00<?, ?it/s]

In [101]:
import numpy as np

arr1 = np.array(ans)
arr2 = np.array(true)

# Вычисляем среднеквадратичную ошибку (MSE)
mse = np.mean((arr1 - arr2)**2)
print("MSE:", mse)

# Вычисляем коэффициент детерминации (R^2)
ss_res = np.sum((arr1 - arr2)**2)
ss_tot = np.sum((arr1 - np.mean(arr1))**2)
r_squared = 1 - (ss_res / ss_tot)
print("R^2:", r_squared)

# Вычисляем среднюю абсолютную ошибку (MAE)
mae = np.mean(np.abs(arr1 - arr2))
print("MAE:", mae)

# Вычисляем корень среднеквадратичной ошибки (RMSE)
rmse = np.sqrt(np.mean((arr1 - arr2)**2))
print("RMSE:", rmse)


MSE: 3729.023092369478
R^2: -50.70981150205793
MAE: 46.57730923694779
RMSE: 61.06572764136589


# Char-LSTM


In [10]:
alp = '1234567890+=-'
pad = 0

In [11]:
vocab = list(alp)

char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = {i:u for i, u in enumerate(vocab)}

In [12]:
def generate_random_number():
    length = random.randint(1, 3)
    if length == 0:
        return 0
    else:
        return random.randint(10**(length-1), 10**length-1)

In [13]:
class NumDataset:
    def __init__(self, length):
        self.length = length
    
    def __len__(self):
        return self.length

    def __getitem__(self, item):
        x = generate_random_number()
        y = generate_random_number()
        z = x + y

        return [char2idx[c]+1 for c in str(z)], [char2idx[c]+1 for c in f'{x}+{y}=']


In [105]:
df = NumDataset(50000)

In [106]:
df.__getitem__(2)

([1, 5, 9], [7, 3, 11, 8, 6, 12])

In [108]:
train_idx, val_idx = train_test_split(range(50000), test_size=0.1, random_state=42)

In [109]:
train_dataset = Subset(df, train_idx)
val_dataset = Subset(df, val_idx)

In [110]:
def collate_fn(items):
    tokens = [torch.tensor(i[1]) for i in items]
    ans = [torch.tensor(i[0]) for i in items]

    max_len = max([len(token) for token in tokens])
    padded_tokens = [torch.nn.functional.pad(token, (0, max_len - len(token)), value=pad) for token in tokens]
    packed_tokens = torch.stack(padded_tokens)

    max_len = 4
    padded_ans = [torch.nn.functional.pad(token, (0, max_len - len(token)), value=pad) for token in ans]
    packed_ans = torch.stack(padded_ans)

    # print(packed_tokens, packed_ans)

    return packed_ans, packed_tokens


In [111]:
train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=8,
        #num_workers=NUM_WORKERS,
        shuffle=True,
        collate_fn=collate_fn,
    )

In [112]:
for batch_idx, (data, target) in enumerate(train_loader):
    print(data.shape, target.shape)
    break


torch.Size([8, 4]) torch.Size([8, 7])


In [113]:
valid_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=8,
        #num_workers=NUM_WORKERS,
        shuffle=False,
        collate_fn=collate_fn,
    )

In [114]:
class MyModel(nn.Module):
   def __init__(self, input_size, hidden_size, output_size, input_size2):
     super().__init__() 
     self.emb = torch.nn.Embedding(input_size, hidden_size, scale_grad_by_freq=True)
     self.rnn = nn.LSTM(hidden_size, hidden_size, batch_first=True, dropout=0.2, num_layers=2)
     self.fc = nn.Linear(hidden_size*1, input_size)

   def forward(self, t):
     t = self.emb(t)
     out, _ = self.rnn(t)
     out = out[:, -4:, :]
     out = self.fc(out)
    
     return out.transpose(1, 2)


In [115]:
input_size = len(vocab)
hidden_size = 256
output_size = len(vocab)
input_size2 = 22 #40


model = MyModel(input_size, hidden_size, output_size, input_size2)
model.to(device)

MyModel(
  (emb): Embedding(13, 256, scale_grad_by_freq=True)
  (rnn): LSTM(256, 256, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=256, out_features=13, bias=True)
)

In [116]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, factor=0.8, patience=5, verbose=True
)

criterion = nn.CrossEntropyLoss()

In [117]:
def train_fn(model, data_loader, optimizer):
    model.train()
    fin_loss = 0
    
    tk0 = tqdm(data_loader, total=len(data_loader))
    for x, t in tk0:
        x = x.to(device)
        t = t.to(device)
        
        model.zero_grad()
        out = model(t)
        loss = criterion(out, x)
        loss.backward()
        optimizer.step()

        fin_loss += loss.item()
        
    return fin_loss / len(data_loader)


In [118]:
def eval_fn(model, data_loader):
    model.eval()
    fin_loss = 0


    fin_preds = 0
    tk0 = tqdm(data_loader, total=len(data_loader))
    with torch.no_grad():
        for x, t in tk0:
          x = x.to(device)
          t = t.to(device)

          out = model(t)
          loss = criterion(out, x)
          
          # fin_preds += mean_squared_error(out, x)
          fin_loss += loss.item()
        
    return fin_loss / len(data_loader) #,  fin_preds / len(data_loader)

In [119]:
from sklearn.metrics import mean_squared_error

In [120]:
for epoch in range(20):

    train_loss = train_fn(model, train_loader, optimizer)
    valid_loss = eval_fn(model, train_loader) # valid_loader
 
    print(
        f"Epoch={epoch}, Train Loss={train_loss}, Evaluation={valid_loss}"
    )

    scheduler.step(valid_loss)

  0%|          | 0/5625 [00:00<?, ?it/s]

  0%|          | 0/5625 [00:00<?, ?it/s]

Epoch=0, Train Loss=1.5355130306773717, Evaluation=1.3212751742680868


  0%|          | 0/5625 [00:00<?, ?it/s]

  0%|          | 0/5625 [00:00<?, ?it/s]

...

  0%|          | 0/5625 [00:00<?, ?it/s]

Epoch=18, Train Loss=0.7469713797489802, Evaluation=0.7497930284261703


  0%|          | 0/5625 [00:00<?, ?it/s]

  0%|          | 0/5625 [00:00<?, ?it/s]

Epoch=19, Train Loss=0.7609598099125756, Evaluation=0.7460494605806138


In [ ]:
!pip install src

In [42]:
torch.cuda.empty_cache()